## Initial setup

Picks GPU, prepares file structure and initializes variables

In [1]:
import tensorflow as tf
import os
import sys

physical_devices = tf.config.list_physical_devices('GPU')
print("Avaliable GPUs:")
for device in physical_devices:
    print(device.name)
print(f"Chosen GPU: {physical_devices[-1].name}")
tf.config.set_visible_devices(physical_devices[-1], 'GPU')

path_common= os.path.dirname(os.path.abspath('')) + "/common/"
sys.path.append(path_common)
print("Current working directory:", os.getcwd())

model, hpo_search_function, hpo_n_epochs_max, hpo_max_trials, hpo_batch_size = [None] * 5

2024-03-19 14:12:04.371163: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 14:12:04.393033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 14:12:04.393048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 14:12:04.393658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 14:12:04.397513: I tensorflow/core/platform/cpu_feature_guar

Avaliable GPUs:
/physical_device:GPU:0
Chosen GPU: /physical_device:GPU:0
Current working directory: /home/slagarp/Master-Thesis/src/main


2024-03-19 14:12:05.319534: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 14:12:05.330896: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 14:12:05.330932: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Pick your model

Available options:

- FCN:                Fully Convultional Neural Network
- ResNet:             Residual Neural Network
- LSTM_FCN:           Long Short Term Memory with Fully Convulutional Network
- Transformer:        ransformer Based model

In [2]:
from ModelFuncs import FCN, ResNet, LSTM_FCN, Transformer
model = FCN()

## Hyper Parameter Optimization

Pick search function and tuning parameters

Available search functions:
- random:               Random Search
- bayesian:             Bayesian Optimization

Available parameters:
- hpo_n_epochs_max
- hpo_max_trials
- hpo_batch_size

In [3]:
from HyperParameterTuning import Tuner

hpo_search_function = "random"

hpo_n_epochs_max = 30
hpo_max_trials = 50
hpo_batch_size = 128

hpo_tuner = Tuner(model, n_epochs_max = hpo_n_epochs_max, max_trials = hpo_max_trials, batch_size = hpo_batch_size, search_function=hpo_search_function, skip_folds = 3)

Run hyper parameter tuning

In [4]:
#%%capture output
hpo_tuner.tune()
#output.show()

Trial 3 Complete [00h 00m 57s]
val_accuracy: 0.2708924114704132

Best val_accuracy So Far: 0.2708924114704132
Total elapsed time: 23h 55m 50s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
128               |128               |filters_block_0
7                 |13                |kernel_size_block_0
64                |128               |filters_block_1
7                 |3                 |kernel_size_block_1
512               |128               |filters_block_2
5                 |13                |kernel_size_block_2
32                |512               |filters_block_3
15                |9                 |kernel_size_block_3
128               |32                |filters_block_4
3                 |9                 |kernel_size_block_4
64                |256               |filters_block_5
13                |11                |kernel_size_block_5
128               |256               |filters_block_6
5                 |7                 |kernel_size_bl

KeyboardInterrupt: 

In [ ]:
hpo_tuner.summary()

Reloading Tuner from ../../results/random/FCN/tuner0.json
Results summary
Results in ../../results/random/FCN
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
filters_block_0: 64
kernel_size_block_0: 9
filters_block_1: 256
kernel_size_block_1: 9
filters_block_2: 512
kernel_size_block_2: 3
filters_block_3: 32
kernel_size_block_3: 13
filters_block_4: 256
kernel_size_block_4: 9
filters_block_5: 64
kernel_size_block_5: 11
filters_block_6: 64
kernel_size_block_6: 9
filters_block_7: 32
kernel_size_block_7: 3
lr: 0.0009731586253939864
Score: 0.1789338082075119

Trial 01 summary
Hyperparameters:
filters_block_0: 32
kernel_size_block_0: 5
filters_block_1: 256
kernel_size_block_1: 5
filters_block_2: 32
kernel_size_block_2: 3
filters_block_3: 256
kernel_size_block_3: 13
filters_block_4: 512
kernel_size_block_4: 5
filters_block_5: 128
kernel_size_block_5: 3
filters_block_6: 512
kernel_size_block_6: 9
filters_block_7: 32
kernel_size_block_7: 

## Train Model

Get best hyper parameters

In [ ]:
tuner_object = hpo_tuner.get_tuner()

tuner_object.reload()

best_hps = tuner_object.get_best_hyperparameters()[0]

best_model = model.build(best_hps)

best_model.summary()



Reloading Tuner from ../../results/random/FCN/tuner0.json
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None, 1)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          640       
                                                                 
 batch_normalization_1 (Bat  (None, None, 64)          256       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, None, 64)          0         
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 

Train model

In [ ]:
from ModelTrainer import ModelTrainer
mt=ModelTrainer(n_epochs_max=10); 
mt.train_es(best_model)

=== Epoch: 0 ===
787/787 [==============================] - 9s 10ms/step - loss: 1.9514 - accuracy: 0.2493
Validation ds:
16/16 [==============================] - 0s 12ms/step - loss: 1.8673 - accuracy: 0.2848
  prep time: 2.9 sec   train time: 9.2 sec
=== Epoch: 1 ===
257/787 [========>.....................] - ETA: 5s - loss: 1.9131 - accuracy: 0.2637

KeyboardInterrupt: 